# Task 1

Solve some of the troublesome problems from [KN1] with Gurobi and other solvers
(soplex, glpsol) and analyze the logs.


The
following linear program provides an example of ill conditioning
and round-off error in the input data:

$$
\begin{array}{rl}
\text{maximize} \;\;&x_1 + x_2 \\
\text{subject to} \; \; &\frac{1}{3}x_1 + \frac{2}{3}x_2 = 1\\
&x_1+2x_2=3 \\
&x_1,x_2 \geq 0.
\end{array}
$$

In [113]:
import gurobipy as gp

def printm(m: "model"):
    for v in m.getVars():
        print(f"{v.varName} = {v.x}") 

In [114]:

# Model 1.1
model = gp.Model("klotz_1")

x1 = model.addVar(lb=0.0, vtype=gp.GRB.CONTINUOUS, name="x1")  
x2 = model.addVar(lb=0.0, vtype=gp.GRB.CONTINUOUS, name="x2")  

model.setObjective(x1 + x2, gp.GRB.MAXIMIZE)

c1_1 = model.addConstr((1/3)*x1 + (2/3)*x2 == 1, "c1_1")
c2 = model.addConstr(x1+2*x2 == 3, "c2")

model.printStats()
model.write("klotz_1.lp")
model.optimize()
printm(model)


Statistics for model 'klotz_1':
  Problem type                : LP
  Linear constraint matrix    : 2 rows, 2 columns, 4 nonzeros
  Variable types              : 2 continuous, 0 integer (0 binary)
  Matrix range                : [3e-01, 2e+00]
  Objective range             : [1e+00, 1e+00]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [1e+00, 3e+00]
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x030fc4dc
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.000000

In [115]:
model.remove(c1_1)
c1_2 = model.addConstr(0.33333*x1+0.66667*x2==1, "c1_2")
model.optimize()
printm(model)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xb3e7b8f2
Coefficient statistics:
  Matrix range     [3e-01, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.000000000e+00
x1 = 1.0
x2 = 1.0


In [116]:
model.remove(c1_2)
c1_3 = model.addConstr(x1+2*x2==3, "c1_3")
model.optimize()
printm(model)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xcd9ba774
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.000000000e+00
x1 = 3.0
x2 = 0.0


The second model gives different results. 
Read about [the Issues and Limitations of Floating-Point Arithmetic](https://docs.python.org/3/tutorial/floatingpoint.html).

$$
\begin{array}{rl}
\text{maximize} \;\;&1\\
\text{subject to} \; \;& -x_1+24x_2\leq 21\\
&x_1\leq 3\\
&x_2 \geq 1.00000008.
\end{array}
$$

In [117]:
# Model 1.1
model = gp.Model("klotz_2")

x1 = model.addVar(vtype=gp.GRB.CONTINUOUS, name="x1")  
x2 = model.addVar(vtype=gp.GRB.CONTINUOUS, name="x2")  

model.setObjective(1, gp.GRB.MAXIMIZE)

c1 = model.addConstr(-x1 + 24*x2 <= 21, "c1")
c2 = model.addConstr(x1 <= 3, "c2")
c3 = model.addConstr(x2 >= 1.00000008, "c3")

model.printStats()
model.write("klotz_2.lp")
model.optimize()

Statistics for model 'klotz_2':
  Problem type                : LP
  Linear constraint matrix    : 3 rows, 2 columns, 4 nonzeros
  Variable types              : 2 continuous, 0 integer (0 binary)
  Matrix range                : [1e+00, 2e+01]
  Objective range             : [0e+00, 0e+00]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [1e+00, 2e+01]
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x5bbed78e
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


The infeasibility is detected by the presolving step.
Let's try to remove it. Parameters can be set in gurobipy in one of the
following ways:
```
m.Params.timeLimit = 100.0
m.Params.timelimit = 100.0
m.Params.TIME_LIMIT = 100.0
m.setParam(GRB.Param.TimeLimit, 100.0)
m.setParam("TimeLimit", 100.0);
```
A list of parameters can be found in this [documentation page](https://docs.gurobi.com/projects/optimizer/en/current/concepts/parameters/groups.html).

In [118]:
if model.status == gp.GRB.status.INF_OR_UNBD:
    # Turn presolve off to determine whether m is infeasible or unbounded
    model.setParam("Presolve", 0) # off
    model.setParam("Method", 0) # -1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex
    model.optimize()
if model.status == gp.GRB.status.OPTIMAL:
    printm(model) 
elif model.status != gp.GRB.status.INFEASIBLE:
    print('Optimization was stopped with infeasible status %d' % model.status)

In [119]:
model.setParam("DualReductions", 0)
model.setParam("InfUnbdInfo",1)
model.optimize()
print(model.FarkasProof)
print(model.FarkasDual)


Set parameter DualReductions to value 0
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
DualReductions  0
InfUnbdInfo  1

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x5bbed78e
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.00s
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   4.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Infeasible model
1.9199999989893968e-06
[1.0, 1.0, -24.0]


# Task 2

Interior point algorithm. 

$$
\begin{array}{rl}
\text{maximize} \;\;&2x_1 + 3x_2 + 2x_3 \\
\text{subject to} \; \; &x_1 + x_2 +2x_3 = 3\\
&x_1,x_2,x_3 \geq 0.
\end{array}
$$

using as starting solution $[x_1, x_2, x_3]=[1, 3/2, 1/4]$.

In [120]:
# %load -r 1-29 src/affine.py
# %% 
from fractions import Fraction
import numpy as np
np.set_printoptions(precision=3, suppress=True)

# %%
def affine_scaling(c, A, b, x_0, alpha=0.5):
    x = x_0
    x_old=np.zeros(len(x))

    while np.linalg.norm(x_old-x)>0.001:
        x_old=x
        D=np.diag(x)
        D1=np.linalg.inv(D)
        
        x_tilde = D1 @ x
        A_tilde = A @ D
        c_tilde = c @ D
        
        P=np.identity(len(x))-A_tilde.T @ np.linalg.inv( A_tilde @ A_tilde.T) @ A_tilde
        p_tilde=P @ c_tilde
        
        theta = np.max([abs(v) for v in p_tilde if v<0])
        
        x_tilde = x_tilde + alpha/theta * p_tilde
        x = D @ x_tilde
        print(x)
    return x


Let's test it on the problem for the slides for which we know the optimal
solution:

In [121]:
# %load -r 30-39 src/affine.py
# %%
c = np.array([1, 2, 0])
A = np.array([[1, 1, 1]])
b = np.array([8])

x_0=np.array([1,3,4])
alpha = 0.5

affine_scaling(c, A, b, x_0, alpha)


[1.046 4.954 2.   ]
[0.934 6.066 1.   ]
[0.724 6.776 0.5  ]
[0.465 7.285 0.25 ]
[0.249 7.626 0.125]
[0.125 7.812 0.062]
[0.063 7.906 0.031]
[0.031 7.953 0.016]
[0.016 7.977 0.008]
[0.008 7.988 0.004]
[0.004 7.994 0.002]
[0.002 7.997 0.001]
[0.001 7.999 0.   ]
[0.    7.999 0.   ]


array([0.   , 7.999, 0.   ])

In [122]:
# %load -r 40-50 src/affine.py
# %%

c = np.array([2, 3, 2])
A = np.array([[1, 1, 2]])
b = np.array([3])

x_0=np.array([1, 3/2, 1/4])
alpha = 0.5

affine_scaling(c, A, b, x_0, alpha)


[0.636 2.114 0.125]
[0.318 2.536 0.073]
[0.159 2.763 0.039]
[0.08  2.881 0.02 ]
[0.04 2.94 0.01]
[0.02  2.97  0.005]
[0.01  2.985 0.002]
[0.005 2.993 0.001]
[0.002 2.996 0.001]
[0.001 2.998 0.   ]
[0.001 2.999 0.   ]
[0. 3. 0.]


array([0., 3., 0.])